In [1]:
import os
import json
from typing import List, Optional
from fb_post_tracker import FbPostTracker
from fb_post_manager import FbPostManager
from fb_api_client import FbApiClient

In [2]:
def load_fb_config(config_file: str) -> dict:
    """Load the Facebook configuration from a JSON file."""
    print(f"Loading Facebook configuration from: {config_file}")
    with open(config_file, "r") as f:
        config = json.load(f)
    print("Facebook configuration loaded successfully")
    return config


def load_credentials(fb_config: dict) -> dict:
    """Load Facebook credentials from environment variables."""
    print("Loading Facebook credentials from environment variables")
    credentials = {}
    try:
        for key in [
            "app_id",
            "app_secret",
            "access_token",
            "page_id",
            "user_id",
            "user_token",
        ]:
            credentials[key] = os.environ[fb_config[key]]
            print(
                f"Loaded {key}: {credentials[key][:5]}..."
            )  # Print first 5 characters for security
    except KeyError as e:
        print(
            f"Error: Environment variable {e} not set. Please set it before running the script."
        )
        exit(1)
    print("Facebook credentials loaded successfully")
    return credentials


def compose_message(post_data: dict) -> str:
    """Compose the full message for the post."""
    print("Composing message for the post")
    message = post_data.get("Post comment", "").strip()
    hashtags = post_data.get("hashtags separated by blank", "").strip()
    mentions = post_data.get("mentions separated by blank", "").strip()

    if hashtags:
        message += f"\n\n{hashtags}"
    if mentions:
        message += f"\n\n{mentions}"

    print(f"Composed message: {message[:50]}...")  # Print first 50 characters
    return message


def get_media_paths(post_data: dict) -> List[str]:
    """Get the list of media paths from the post data."""
    print("Getting media paths")
    media_sources = post_data.get(
        "Media Souce links separated by comma", ""
    )  # Note the typo in "Souce"
    paths = [path.strip() for path in media_sources.split(",") if path.strip()]
    print(f"Media paths: {paths}")
    return paths


def get_media_titles(post_data: dict) -> List[str]:
    """Get the list of media titles from the post data."""
    print("Getting media titles")
    media_titles = post_data.get("titles of media source separated by comma", "")
    titles = [title.strip() for title in media_titles.split(",") if title.strip()]
    print(f"Media titles: {titles}")
    return titles

In [3]:
# Load Facebook configuration
config_file = r"C:\Users\manue\Documents\GitHub007\sn-libraries\config_files\FB_JK_JK Travel_JK Travel_config.json"
fb_config = load_fb_config(config_file)
credentials = load_credentials(fb_config)

# Set up Google Sheets configuration
account_id = "JK"
spreadsheet_id = "1wrvG3wmptA76kywbVe1gy5as-ALDzmebLvqoxWIw9mw"
print(
    f"Google Sheets configuration - Account ID: {account_id}, Spreadsheet ID: {spreadsheet_id}"
)

# Initialize the necessary components
print("Initializing API client, post tracker, and post manager")
api_client = FbApiClient(credentials)
tracker = FbPostTracker(account_id, spreadsheet_id)
fb_post_manager = FbPostManager(api_client)

Loading Facebook configuration from: C:\Users\manue\Documents\GitHub007\sn-libraries\config_files\FB_JK_JK Travel_JK Travel_config.json
Facebook configuration loaded successfully
Loading Facebook credentials from environment variables
Loaded app_id: 16318...
Loaded app_secret: 7df81...
Loaded access_token: EAAXM...
Loaded page_id: 37995...
Loaded user_id: 37995...
Loaded user_token: EAAXM...
Facebook credentials loaded successfully
Google Sheets configuration - Account ID: JK, Spreadsheet ID: 1wrvG3wmptA76kywbVe1gy5as-ALDzmebLvqoxWIw9mw
Initializing API client, post tracker, and post manager


In [4]:
def get_next_post():
    # Get the next unpublished post
    print("Getting next unpublished post")
    post_data = tracker.get_next_unpublished_post()
    if not post_data:
        print("No posts to publish")
        return None

    print(f"Post data retrieved: {post_data}")

    post_type = post_data.get("Type", "").lower()
    print(f"Post type: {post_type}")

    message = compose_message(post_data)
    media_paths = get_media_paths(post_data)
    media_titles = get_media_titles(post_data)

    return post_data, post_type, message, media_paths, media_titles

In [5]:
# Call the function
result = get_next_post()
if result:
    post_data, post_type, message, media_paths, media_titles = result
    print("Ready to publish post")
    print(f"Post data: {post_data}")
    print(f"Post type: {post_type}")
    print(f"Message: {message}")
    print(f"Media paths: {media_paths}")
    print(f"Media titles: {media_titles}")
else:
    print("No post to publish")

Getting next unpublished post
Post data retrieved: {'Ref #': '1', 'Subject': 'Japan Imperial Palace', 'Type': 'single photo', '# of media': '1', 'Media Souce links separated by comma': 'photo100-japan-Imperial palace.jpeg', 'titles of media source separated by comma': 'Japan Imperial Palace', 'Post comment': 'aaa', 'hashtags separated by blank': '#travel #travelaroundtheworld', 'mentions separated by blank': '@disfrutandohuelva', 'location': 'japan', 'shared to group (Groups ids)': 'group 1, group 2', 'Published? Y/N': '', 'Date and Time': '', 'ID (str.)': '', 'row_index': 3}
Post type: single photo
Composing message for the post
Composed message: aaa

#travel #travelaroundtheworld

@disfrutandohu...
Getting media paths
Media paths: ['photo100-japan-Imperial palace.jpeg']
Getting media titles
Media titles: ['Japan Imperial Palace']
Ready to publish post
Post data: {'Ref #': '1', 'Subject': 'Japan Imperial Palace', 'Type': 'single photo', '# of media': '1', 'Media Souce links separated 

In [9]:
print(f"Attempting to publish {post_type} post")
try:
    if post_type == "text":
        post_id = fb_post_manager.publish_text_post(credentials["page_id"], message)
    elif post_type == "single photo":
        if media_paths:
            post_id = fb_post_manager.publish_photo_post(
                credentials["page_id"], message, media_paths[0]
            )
        else:
            raise ValueError("No media path provided for single photo post")
    elif post_type == "multiple photo":
        post_id = fb_post_manager.publish_multi_photo_post(
            credentials["page_id"], message, media_paths
        )
    elif post_type == "video":
        if media_paths:
            title = media_titles[0] if media_titles else None
            post_id = fb_post_manager.publish_video_post(
                credentials["page_id"], message, media_paths[0], title
            )
        else:
            raise ValueError("No media path provided for video post")
    elif post_type == "reel":
        if media_paths:
            title = media_titles[0] if media_titles else None
            post_id = fb_post_manager.publish_reel(
                credentials["page_id"], message, media_paths[0], title
            )
        else:
            raise ValueError("No media path provided for reel post")
    else:
        print(f"Unknown post type: {post_type}")

    if post_id:
        print(f"Successfully published {post_type} post with ID: {post_id}")
        print("Updating spreadsheet with published information")
        tracker.mark_post_as_published(post_data["row_index"], post_id)
        tracker.add_post_to_published_log(post_data)
    else:
        print(f"Failed to publish {post_type} post")
        print("Updating spreadsheet with failure status")
        tracker.update_post_status(post_data["row_index"], "Failed")

except Exception as e:
    print(f"Error publishing {post_type} post: {str(e)}")
    print("Updating spreadsheet with error status")
    tracker.update_post_status(post_data["row_index"], "Error")

Attempting to publish single photo post
Error publishing post with photo: [Errno 2] No such file or directory: 'photo100-japan-Imperial palace.jpeg'
Failed to publish single photo post
Updating spreadsheet with failure status
